# Lista 13 (8 pkt.)

We wszystkich poniższych zadaniach będziemy używać **OpenSSL** https://www.openssl.org/.

## Krótki tutorial:

### generowanie klucza prywatnego
```
openssl genpkey -algorithm RSA -pkeopt rsa_keygen_bits:2048 -pkeyopt rsa_keygen_pubexp:65537 -out privkey.pem
```
```-algorithm``` wybiera algorytm do generacji klucza, tu ```RSA```

```-pkeyopt rsa_keygen_bits:2048``` ustala długość klucza, tu ```2048``` bitów

```-pkeyopt rsa_keygen_pubexp:65537``` ustala wartość $e$, tu ```65537```

```-out privkey.pem``` zapisuje klucz prywatny w pliku ```privkey.pem```
### generowanie klucza publicznego
```
openssl pkey -in privkey.pem -pubout -out pubkey.pem
```
```-in privkey.pem``` wczytuje plik z kluczem prywatnym

```-pubout``` ustala, że chcemy uzyskać klucz publiczny

```-out pubkey.pem``` zapisuje klucz publiczny w pliku ```pubkey.pem```

### Diffy-Helman
- generowanie publicznych parametrów $p$ i $g$
```
openssl genpkey -genparam -algorithm DH -out dh_params.pem
```
- generowanie klucza prywatnego $a$
```
openssl genpkey -paramfile dh_params.pem -out privkey.pem
```
- generowanie klucza publicznego $g^a$
```
openssl pkey -in privkey.pem -pubout -out pubkey.pem
```
- generowanie wspólnego sekretu $k=(g^b)^a=(g^a)^b$
```
openssl pkeyutl -derive -inkey privkey_A.pem -peerkey pubkey_B.pem -out secret.bin
```

### wyświetlanie w przyjaznej formie
- klucz prywatny
```
openssl pkey -in privkey.pem -text -noout
```
- klucz publiczny
```
openssl pkey -pubin -in pubkey.pem -text -noout
```
- certyfikat
```
openssl x509 -in user.crt -text -noout
```
- parametry Diffiego-Helmana
```
openssl pkeyparam -in dh_params.pem -text -noout
```
### generowanie żądania popisania certyfikatu
```
openssl req -new -key privkey.pem -out req.csr
```
```-key privkey.pem``` wskazuje plik z naszym kluczem prywatnym

```-out req.csr``` zapisuje żądanie certyfikatu do pliku ```req.csr``

po wpisaniu tej komendy pojawi się seria pytań o dane do certyfikatu, można wpisać jakieś rzeczy typu PL, Cracow, UJ itd., można też wpisać '.' (kropkę) i w ten sposób pominąć dane pole

Uwaga: aby certyfikat przechodził później weryfikację bez problemu, dane wpisane w certyfikat użytkownika, i dane wpisane w certyfikat **CA** nie mogą być identyczne.

### generowanie samopodpisanego certyfikatu przez **CA**
```
openssl req -x509 -new -nodes -key rootkey.pem -sha256 -days 1024 -out root.crt
```

```-key rootkey.pem``` wskazuje klucz prywatny **CA**

```-sha256``` wybiera funkcję hashującą używaną do podpisu, tu **SHA 256**

```-days 1024``` ustala termin ważności certyfikatu

```-out root.crt``` zapisuje certyfikat do pliku ```root.crt```

### wygenerowanie i podpisanie certyfiaktu przez **CA** na podstawie żądania
```
openssl x509 -req -in req.csr -CA root.crt -CAkey rootkey.pem -CAcreateserial -out user.crt -days 500 -sha256
```
```-req -in req.csr``` wskazuje żądanie

```-CA root.crt``` wskazuje certyfikat **CA**

```-CAcreateserial``` tworzy numer seryjny cerrtyfikatu

```-out user.crt``` zapisuje podpisany certyfikat do pliku ```user.crt```

```-days 500``` ustala termin ważności certyfikatu, tu ```500``` dni

```-sha256``` wybiera funkcję hashującą używaną do podpisu, tu **SHA 256**
### weryfikacja certyfikatu
```
openssl verify -CAfile root.crt user.crt
```
```-CAfile root.crt``` wskazuje certyfikat **CA** na podstawie którego weryfikujemy certyfiakt użytkownika

```user.crt``` certyfikat, który weryfikujemy
### wydobycie klucza publicznego użytkownika z jego certyfikatu
```
openssl x509 -pubkey -in user.crt -noout > pubkey.pem
```
```-pubkey``` wskazuje, że wyciągamy klucz publiczny

```-in user.crt``` wskazuje plik z certyfikatem

```> pubkey.pem``` zapisuje uzyskany klucz do pliku ```pubkey.pem```

### szyfrowanie pliku za pomocą szyfru asymetrycznego
```
openssl pkeyutl -encrypt -in file.txt -pubin -inkey pubkey.pem -out file.enc
```
```-encrypt``` włącza tryb szyfrowania

```-in file.txt``` wskazuje plik do zaszyfrowania

```-pubin -inkey pubkey.pem``` wskazuje klucz publiczny, którym szyfrujemy

```-out file.enc``` zapsiuje zaszyfrowany plik do pliku ```file.enc```
### deszyfrowanie pliku za pomocą szyfru asymetrycznego
```
openssl pkeyutl -decrypt -in file.enc -inkey privkey.pem -out file.txt
```
```-decrypt``` włącza tryb deszyfrowania

```-in file.enc``` wskazuje plik do deszyfrowania

```-inkey privkey.pem``` wskazuje klucz prywatny, którym deszyfrujemy

```-out fil.txt``` zapsiuje odszyfrowany plik do pliku ```file.txt```

### szyfrowanie pliku za pomocą szyfru symetrycznego
- na podstawie hasła
```
openssl enc -aes-256-cbc -p -pbkdf2 -in file.txt -out file_enc.bin
```
pojawi się zapytanie o hasło, można też dodać ```-pass file:key_pass.pem``` aby pobrać hasło z pliku ```key_pass.pem```

```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```--pbkdf2``` ustala metodę używaną przy generowaniu klucza

```-in file.txt``` plik do zaszyfrowania

```file_enc.bin``` zaszyfrowany plik
- na podstawie klucza
```
openssl enc -aes-256-cbc -K XXXXXXXXX -iv XXXXXXXX -p -in file.txt -out file_enc.bin
```
```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-K XXXXXXXXX``` ustala klucz na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-iv XXXXXXXXX``` ustala wektor inicjujący na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```-in file.txt``` plik do zaszyfrowania

```file_enc.bin``` zaszyfrowany plik

### deszyfrowanie pliku za pomocą szyfru symetrycznego
- na podstawie hasła
```
openssl enc -aes-256-cbc -d -p -pbkdf2 -in file_enc.bin -out file.txt
```
pojawi się zapytanie o hasło, można też dodać ```-pass file:key_pass.pem``` aby pobrać hasło z pliku ```key_pass.pem```

```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-d``` wybiera tryb deszyfracji

```-pass file:key_pass.pem``` wybiera deszyfrowanie na podstawie klucza wygenerowanego z hasła znajdującego się w pliku ```key_pass.pem```

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```-pbkdf2``` ustala funkcję hashującą używaną przy generowaniu klucza

```-in file_enc.bin``` plik do deszyfrowania

```file.txt``` odszyfrowany plik
- na podstawie klucza
```
openssl enc -aes-256-cbc -d -K XXXXXXXXX -iv XXXXXXXX -p -in file.enc -out file.txt
```
```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-d``` wybiera tryb deszyfracji

```-K XXXXXXXXX``` ustala klucz na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-iv XXXXXXXXX``` ustala wektor inicjujący na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```-in file.enc``` plik do odszyfrowania

```file.txt``` odszyfrowany plik

### generowanie podpisu elektronicznego
```
openssl dgst -sha1 -sign privkey.pem -out signature.bin file.txt
```
```-sha1``` wybiera funkcję hashującą **SHA 1**

```-sign privkey.pem``` podpisuje kluczem prywatnym ```privkey.pem```

```-out signature.bin``` zapisuje podpis do pliku binarnego ```signature.bin```

```file.txt``` podpisywany plik
### weryfikacja podpisu elektronicznego
```
openssl dgst -sha1 -verify pubkey.pem -signature signature.bin file.txt
```
```-sha1``` wybiera funkcję hashującą **SHA 1**

```-verify pubkey.pem``` weryfikuje podpis kluczem publicznm ```pubkey.pem```

```-signature signature.bin``` wskazuje plik z podpisem do weryfikacji

```file.txt``` plik, którego podpis weryfikujemy

### generowanie pseudo-losowych bajtów
```
openssl rand -base64 32 -out symkey.pem
```
- ```-base64``` ustala kodowanie na base64

- ```32``` liczba bajtów do wygenerowania

- ```-out symkey.pem``` zapisuje ciąg bajtów do symkey.pem


## Zadanie 1 (1 pkt.)

Zdeszyfruj plik **AESencryptedCBC.enc** za pomocą klucza:

8541F781259ADA8631F18A9A8E97771BFE395CF7ACBE5F77

oraz wektora inicjującego:

99B54E58FF44F7599465E6888A3C6C1E

w trybie **CBC** z kluczem **192** bitowym.

In [2]:
!openssl enc -aes-192-cbc -d -K 8541F781259ADA8631F18A9A8E97771BFE395CF7ACBE5F77 -iv 99B54E58FF44F7599465E6888A3C6C1E -p -in ./AesEncrypted.enc

salt=9C4FD70DFE7F0000
key=8541F781259ADA8631F18A9A8E97771BFE395CF7ACBE5F77
iv =99B54E58FF44F7599465E6888A3C6C1E
Juliusz Słowacki

Hymn o zachodzie słońca na morzu



Smutno mi, Boże! Dla mnie na zachodzie
Rozlałeś tęczę blasków promienistą;
Przede mną gasisz w lazurowej wodzie
    Gwiazdę ognistą…
Choć mi tak niebo Ty złocisz i morze,
    Smutno mi, Boże!
Jak puste kłosy z podniesioną głową,
Stoję rozkoszy próżen i dosytu…
Dla obcych ludzi mam twarz jednakową,
    Ciszę błękitu.
Ale przed Tobą głąb serca otworzę:
    Smutno mi, Boże!
Jako na matki odejście się żali
Mała dziecina, tak ja płaczu bliski,
Patrząc na słońce, co mi rzuca z fali
    Ostatnie błyski,
Choć wiem, że jutro błyśnie nowe zorze,
    Smutno mi, Boże!
Dzisiaj na wielkiem morzu obłąkany,
Sto mil od brzegu i sto mil przed brzegiem,
Widziałem lotne w powietrzu bociany
    Długim szeregiem.
Żem je znał kiedyś na polskim ugorze,
    Smutno mi, Boże!
Żem często dumał nad mogiłą ludzi,
Żem nie znał prawie rodzinnego domu,
Że

## Zadanie 2 (1 pkt.)

Zaszyfruj plik **kamien.txt** za pomocą hasła w trybie **OFB** z kluczem **256** bitowym.

In [4]:
!openssl rand -hex 32
!openssl rand -hex 16

c6494d91625f87ee9c7b2a18328a7a3ce38a64bd76ba5e54915c23a7f52f7308
f00d6602d062ebb7765f41cdea61e60a


In [5]:
!openssl enc -aes-256-ofb -K c6494d91625f87ee9c7b2a18328a7a3ce38a64bd76ba5e54915c23a7f52f7308 -iv f00d6602d062ebb7765f41cdea61e60a -p -pbkdf2 -in kamien.txt -out kamien.enc

salt=8C2A3E75FF7F0000
key=C6494D91625F87EE9C7B2A18328A7A3CE38A64BD76BA5E54915C23A7F52F7308
iv =F00D6602D062EBB7765F41CDEA61E60A


In [6]:
!openssl enc -aes-256-ofb -d -K c6494d91625f87ee9c7b2a18328a7a3ce38a64bd76ba5e54915c23a7f52f7308 -iv f00d6602d062ebb7765f41cdea61e60a -p -in kamien.enc


salt=DCC7BA70FE7F0000
key=C6494D91625F87EE9C7B2A18328A7A3CE38A64BD76BA5E54915C23A7F52F7308
iv =F00D6602D062EBB7765F41CDEA61E60A
Ja tylko jestem kamieniem
I jako niemowlę tu,
Objęty matki ramieniem,
Cichego używam snu.

Ledwie świt ducha półsenny
Mnie z łona martwości zwie
I w mojej piersi kamiennej
Poczuciem istnienia tchnie.

Jeszczem związany łańcuchem
W szeregu bezwiednych brył
Z ogólnym natury duchem,
Z kolebką bezwiednych sił.

Jednak przeczuwam powoli,
Że zacznie pierś moja bić,
I marzę o ludzkiej doli,
Chcę kochać, cierpieć i żyć!

I wierzę, iż ten, co kruszy
Dziś moję powłokę, grom
Otworzy dla mojej duszy
Wspanialszy cielesny dom.

I witam z dreszczem rozkoszy
Każdy zadany mi gwałt -
W niszczeniu, co mnie rozproszy,
Zgadując przyszłości kształt.

Tam, gdzie się boleść zaczyna,
Bezwiednej martwości kres!
A nad nią boska kraina,
Gdzie miłość wykwita z łez.

Przez wszystkie śmierci ogniwa
Postępu ciągnie się nić -
Ból samowiedzę zdobywa:
Chcę cierpieć, kochać i żyć!


Adam Asnyk


## Zadanie 3 (1 pkt.)

Stwórz dwa foldery, **Alice** oraz **Bob**, i przeprowadź protokół uzgadniania klucza za pomocą metody Diffiego-Helmana pomiędzy tymi folderami. Gdy uzsykasz w obu folderach plik **secret.bin**, który jest plikiem binarnym zawierającym uzgodniony ciąg bitów pomiędzy Alicją i Bobem, zaszyfruj plik **rozwijajac_rilkego.txt** w folderze Alicji podając jako hasło plik **secret.bin** a następnie zdeszyfruj go w folderze Boba za pomocą jego pliku **secret.bin**. Użyj AES 256 w dowolnym trybie.

In [38]:
!mkdir Alice Bob

!openssl genpkey -genparam -algorithm DH -out dh_params.pem
!cp dh_params.pem Alice/
!mv dh_params.pem Bob/

!openssl genpkey -paramfile ./Alice/dh_params.pem -out ./Alice/privkey_A.pem
!openssl pkey -in ./Alice/privkey_A.pem -pubout -out ./Alice/pubkey_A.pem

!openssl genpkey -paramfile ./Bob/dh_params.pem -out ./Bob/privkey_B.pem
!openssl pkey -in ./Bob/privkey_B.pem -pubout -out ./Bob/pubkey_B.pem

!cp Alice/pubkey_A.pem Bob/
!cp Bob/pubkey_B.pem Alice/


.....................................+......................................................+..............................................................................................................................................................................................................................................................................................................+....................................+.+................................................+......................+.......................................................................................................................................................................+...........................................................................................................................+..............+.........................................................................................................................................................+......................+.....+...

In [39]:
!openssl pkeyutl -derive -inkey ./Alice/privkey_A.pem -peerkey ./Alice/pubkey_B.pem -out ./Alice/secret.bin
!openssl pkeyutl -derive -inkey ./Bob/privkey_B.pem -peerkey ./Bob/pubkey_A.pem -out ./Bob/secret.bin

In [40]:
!openssl enc -aes-256-cbc -p -pbkdf2 -in rozwijajac_rilkego.txt -out ./Bob/rozwijac_rilkego.enc -pass file:./Alice/secret.bin

salt=14D621872D99906D
key=5E9B49A052954EF66F621E3C1A9FB2E60761EB9491A7A1FEB8B082816B638244
iv =247E1B9CBACB21AA659A70BF8EEED26E


In [41]:
!openssl enc -d -aes-256-cbc -p -pbkdf2 -in ./Bob/rozwijac_rilkego.enc -pass file:./Bob/secret.bin


salt=14D621872D99906D
key=5E9B49A052954EF66F621E3C1A9FB2E60761EB9491A7A1FEB8B082816B638244
iv =247E1B9CBACB21AA659A70BF8EEED26E
W złożonych dłoniach
Strzępy ciszy
Co mogły być modlitwą
Niewiele, ale i to stracisz, Panie
Gdy one znikną
I zachwyt na siatkówce oka
Nietrwały w końcu zapis
Twój hymn dla Twego dzieła, Panie
To także stracisz
I natarczywe słowo próśb
Proście, a będzie Wam dane
Co ucho Twe pieściły, Panie
Tak, także ustaną
I moje miejsce w Twoim domu
Wygniecione przez stopy
Spojrzysz niby przypadkiem, Panie
A tam już tylko kurzu dotyk

Jacek Cygan


## Zadanie 4 (1 pkt.)

Klikając w kłodkę w przeglądarce internetowej otwórz certyfikaty następujących stron:

- Wikipedia
- Google
- Strona tego kursu kryptografii

Odczytaj z tych certyfikatów następujące informacje:
- data ważności certyfikatu
- rodzaj używanego algorytmu asymetrycznego
- klucz publiczny
- rodzaj używanego algorytmu do podpisu

### Wikipedia:

![image.png](./imgs/wiki1.png)
![image.png](./imgs/wiki2.png)
![image.png](./imgs/wiki3.png)

### Google

![image.png](./imgs/google.png)

Ten kurs: (brak certyfikatu)

![image.png](./imgs/kurs.png)

## Zadanie 5 (1 pkt.)

Spróbuj zaszyfrować plik bajka.txt za pomocą RSA. Co się dzieje?

In [22]:
!openssl genpkey -algorithm RSA -pkeyopt rsa_keygen_bits:2048 -pkeyopt rsa_keygen_pubexp:65537 -out privkey.pem


.........+....+.....+......+...............+...+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*........+.+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*..+.+...+.....+.+.....+.+.....+....+..+....+...+...+...............+...+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
...+......+............+...+.......+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.+..+...+.......+..+.+...+..............+.......+......+.....+.......+..+....+.........+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*...+...+...+....+...+..+.......+...........+....+........+...+.............+...............+..+............+.............+..+.+.....+......+...............+.+..+.+..+....+...+........+...+.+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [23]:
!openssl pkey -in privkey.pem -pubout -out pubkey.pem
!openssl pkeyutl -encrypt -in bajka.txt -pubin -inkey pubkey.pem


Public Key operation error
40B76A5FF77A0000:error:0200006E:rsa routines:ossl_rsa_padding_add_PKCS1_type_2_ex:data too large for key size:crypto/rsa/rsa_pk1.c:129:


## Zadanie 6 (1 pkt.)

Sprawdź, który z certyfikatów **ToJa1.crt**, **ToJa2.crt** czy **ToJa3.crt** jest prawdziwy. Certyfikat zaufanej trzeciej strony to **root.crt**.

Następnie wyekstrahuj klucz publiczny z prawdziwego certyfikatu i sprawdź, który z plików **plik1.txt**, **plik2.txt** czy **plik3.txt**, zostały podpisane przez właściciela certyfikatu. Podpisy plików znajdują się odpowiednio w plikach **signature1.bin**, **signature2.bin** i **signature3.bin**. Pliki podpisano za pomocą **SHA 1**.

In [27]:
!openssl verify -CAfile root.crt ToJa1.crt


C = Pl, ST = KRK, O = UJ
error 7 at 0 depth lookup: certificate signature failure
error ToJa1.crt: verification failed
4037E20D02750000:error:02000068:rsa routines:ossl_rsa_verify:bad signature:crypto/rsa/rsa_sign.c:430:
4037E20D02750000:error:1C880004:Provider routines:rsa_verify:RSA lib:providers/implementations/signature/rsa_sig.c:774:
4037E20D02750000:error:06880006:asn1 encoding routines:ASN1_item_verify_ctx:EVP lib:crypto/asn1/a_verify.c:219:


In [26]:
!openssl verify -CAfile root.crt ToJa2.crt

ToJa2.crt: OK


In [28]:
!openssl verify -CAfile root.crt ToJa3.crt


C = PL, ST = KRK, O = Internet Widgits Pty Ltd
error 7 at 0 depth lookup: certificate signature failure
error ToJa3.crt: verification failed
4037AC53BD720000:error:02000068:rsa routines:ossl_rsa_verify:bad signature:crypto/rsa/rsa_sign.c:430:
4037AC53BD720000:error:1C880004:Provider routines:rsa_verify:RSA lib:providers/implementations/signature/rsa_sig.c:774:
4037AC53BD720000:error:06880006:asn1 encoding routines:ASN1_item_verify_ctx:EVP lib:crypto/asn1/a_verify.c:219:


In [29]:
!openssl x509 -pubkey -in ToJa2.crt -noout > validCertPubKey.pem

In [31]:
!openssl dgst -sha1 -verify validCertPubKey.pem -signature signature1.bin plik1.txt
!openssl dgst -sha1 -verify validCertPubKey.pem -signature signature2.bin plik2.txt
!openssl dgst -sha1 -verify validCertPubKey.pem -signature signature3.bin plik3.txt


Verified OK
Verification failure
40A73A7C727D0000:error:0200008A:rsa routines:RSA_padding_check_PKCS1_type_1:invalid padding:crypto/rsa/rsa_pk1.c:75:
40A73A7C727D0000:error:02000072:rsa routines:rsa_ossl_public_decrypt:padding check failed:crypto/rsa/rsa_ossl.c:598:
40A73A7C727D0000:error:1C880004:Provider routines:rsa_verify:RSA lib:providers/implementations/signature/rsa_sig.c:774:
Verification failure
40A7DF190B7B0000:error:0200008A:rsa routines:RSA_padding_check_PKCS1_type_1:invalid padding:crypto/rsa/rsa_pk1.c:75:
40A7DF190B7B0000:error:02000072:rsa routines:rsa_ossl_public_decrypt:padding check failed:crypto/rsa/rsa_ossl.c:598:
40A7DF190B7B0000:error:1C880004:Provider routines:rsa_verify:RSA lib:providers/implementations/signature/rsa_sig.c:774:


## Zadanie 7 (2 pkt.)

W tym zadaniu przećwiczymy bardziej pełny schemat komunikacji. Stwórz trzy foldery **Alice**, **Bob**, **CA**. Alicja będzie nadawcą pliku, Bob odbiorcą a CA to zaufana trzecia strona.

Użyj następujących algorytmów:
- RSA 2048 e=65537
- SHA 256
- AES 256 CBC z kluczem generowanym z hasła

1. Każda ze stron generuje swoje pary klucz prywatny **privkey_Alice.pem** i klucz publiczny **pubkey_Alice.pem**.
2. W ogólości klucz prywatny należy zaszyfrować jakimś szyfrem symetrycznym, np. **AES** aby go bezpiecznie przechowywać, tu dla ułatwienia pomieniemy ten krok.
3. Zaufana trzecia strona **CA** tworzy swój certyfikat, podpisany przez samą siebie **root.crt**, który jest ogłaszany publicznie, tzn. trafia do Alicji i Boba.
4. Następnie Alicja i Bob muszą uzyskać certyfikaty podpisane przez **CA**. W związku z tym każde z nich tworzy rządanie certyfikatu **Alice_req.csr**.
5. Alicja i Bob wysyłają swoje żądania do **CA**, które po zweryfikowaniu Alicji i Boba podpisuje je. Podpisane certyfikaty Alicji i Boba sa ogłaszane publicznie.
6. Alicja bierze certyfikat Boba weryfikuje go za pomocą certyfikatu **CA** (**root.crt**). Jeśli certyfikat jest poprawny, wyciąga z niego klucz publiczny Boba i zapisuje do pliku **pubkey_Bob.pem**.
7. Alicja generuje losowe hasło (**sympass.pem**) z którego będzie generowany klucz symetryczny, które będzie chciała przekazać Bobowi.
8. Alicja szyfruje hasło **sympass.pem** za pomocą klucza publicznego Boba **pubkey_Bob.pem** otrzymując **sympass.enc**.
7. Alicja podpisuje zaszyfrowane hasło poprzez zhashowanie go i zaszyfrowanie za pomocą swojego klucza prywatnego **privkey_ALice.pem**. Podpis zapisuje w pliku **signature.bin**.
8. Alicja wysyła do Boba zaszyfrowane hasło **sympass.enc** oraz jego podpis **signature.bin**.
9. Bob weryfikuje certyfikat Alicji, wyciąga z niego jej klucz publiczny, deszyfruje otrzymane hasło **sympass.enc** i sprawdza podpis.
10. Alicja szyfruje plik, który chce przekazać Bobowi za pomocą hasła **sympass.pem** i wysyła zaszyfrowany plik do Boba.
11. Bob deszyfruje otrzymany plik za pomocą **sympass.pem**.

In [1]:
!mkdir A B CA

mkdir: cannot create directory ‘CA’: File exists



1. Każda ze stron generuje swoje pary klucz prywatny **privkey_Alice.pem** i klucz publiczny **pubkey_Alice.pem**.

In [2]:
!openssl genpkey -algorithm RSA -pkeyopt rsa_keygen_bits:2048 -pkeyopt rsa_keygen_pubexp:65537 -out A/privkey_A.pem
!openssl pkey -in A/privkey_A.pem -pubout -out A/pubkey_Alice.pe
!openssl genpkey -algorithm RSA -pkeyopt rsa_keygen_bits:2048 -pkeyopt rsa_keygen_pubexp:65537 -out B/privkey_B.pem
!openssl pkey -in B/privkey_B.pem -pubout -out B/pubkey_Bob.pem

...........+...+..+...+.......+........+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*......+.+...+..+............+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*....+....+.....+.......+.....+......+..................+..........+.....+.......+..+.......+...+...+..+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
....+..............+.+.....+.+......+..+.......+......+.....+.......+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.+..+.............+..+....+.........+......+...+...+...+....................+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*...+..........+...+...........+.+.....+...+............+.............+.........+..+..................+..........+........+.......+..+...+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
....+...+....+..+.........+....+..+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*..............+.+...+.........+..+.+.....+....+.....+..

3. Zaufana trzecia strona **CA** tworzy swój certyfikat, podpisany przez samą siebie **root.crt**, który jest ogłaszany publicznie, tzn. trafia do Alicji i Boba.

In [3]:
!openssl genpkey -algorithm RSA -pkeyopt rsa_keygen_bits:2048 -pkeyopt rsa_keygen_pubexp:65537 -out CA/rootkey.pem
!openssl req -x509 -new -nodes -key CA/rootkey.pem -sha256 -days 1024 -out root.crt -subj "/C=PL/ST=PL/L=Cracow/O=CA/OU=IT/CN=CA"

...............+......+....+...+.....+....+.....+.+............+..+.+..............+...+.+.....+.........+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*..+...+.+......+.....+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*..........+......+...+.........+........+....+...+......+..+...+....+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
......+.......+.....+.........+...+.......+.........+......+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*...+..+.........+....+.....+.+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.+.+......+......+..+.+......+...+..+.........+.+...+......+.........+..+.+..................+.........+...+..+.+..+.........+.........+......+...+............+.........+.+...+......+...........+.+..+...+....+.....+...+...........................+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


4. Następnie Alicja i Bob muszą uzyskać certyfikaty podpisane przez **CA**. W związku z tym każde z nich tworzy rządanie certyfikatu **Alice_req.csr**.

In [4]:
!openssl req -new -key A/privkey_A.pem -out A/A_req.csr -subj "/C=PL/ST=PL/L=Cracow/O=Alice/OU=IT/CN=A"
!openssl req -new -key B/privkey_B.pem -out B/B_req.csr -subj "/C=PL/ST=PL/L=Cracow/O=Bob/OU=IT/CN=B"



5. Alicja i Bob wysyłają swoje żądania do **CA**, które po zweryfikowaniu Alicji i Boba podpisuje je. Podpisane certyfikaty Alicji i Boba sa ogłaszane publicznie.

In [5]:
!openssl x509 -req -in A/A_req.csr -CA root.crt -CAkey CA/rootkey.pem -CAcreateserial -out A/A.crt -days 500 -sha256
!openssl x509 -req -in B/B_req.csr -CA root.crt -CAkey CA/rootkey.pem -CAcreateserial -out B/B.crt -days 500 -sha256


Certificate request self-signature ok
subject=C = PL, ST = PL, L = Cracow, O = Alice, OU = IT, CN = A
Certificate request self-signature ok
subject=C = PL, ST = PL, L = Cracow, O = Bob, OU = IT, CN = B



6. Alicja bierze certyfikat Boba weryfikuje go za pomocą certyfikatu **CA** (**root.crt**). Jeśli certyfikat jest poprawny, wyciąga z niego klucz publiczny Boba i zapisuje do pliku **pubkey_Bob.pem**.

In [6]:
!openssl verify -CAfile root.crt B/B.crt
!openssl x509 -pubkey -in B/B.crt -noout > A/pubkey_B.pem

B/B.crt: OK



7. Alicja generuje losowe hasło (**sympass.pem**) z którego będzie generowany klucz symetryczny, które będzie chciała przekazać Bobowi.

In [7]:
!openssl rand -base64 -out A/sympass.pem 32

8. Alicja szyfruje hasło **sympass.pem** za pomocą klucza publicznego Boba **pubkey_Bob.pem** otrzymując **sympass.enc**.


In [8]:
!openssl pkeyutl -encrypt -pubin -inkey A/pubkey_B.pem -in A/sympass.pem -out A/sympass.enc


7. Alicja podpisuje zaszyfrowane hasło poprzez zhashowanie go i zaszyfrowanie za pomocą swojego klucza prywatnego **privkey_ALice.pem**. Podpis zapisuje w pliku **signature.bin**.


In [9]:
!openssl dgst -sha256 -sign A/privkey_A.pem -out A/signature.bin A/sympass.enc

8. Alicja wysyła do Boba zaszyfrowane hasło **sympass.enc** oraz jego podpis **signature.bin**.
9. Bob weryfikuje certyfikat Alicji, wyciąga z niego jej klucz publiczny, deszyfruje otrzymane hasło **sympass.enc** i sprawdza podpis.


In [10]:
!openssl verify -CAfile root.crt A/A.crt


A/A.crt: OK


Bob wyciaga klucz publiczny z certyfikatu

In [11]:
!openssl x509 -pubkey -in A/A.crt -noout > B/pubkey_A.pem

Sprawdzanie podpisu

In [12]:
!openssl dgst -sha256 -verify B/pubkey_A.pem -signature A/signature.bin A/sympass.enc


Verified OK


Deszyfracja hasłą sympass.enc

In [13]:
!openssl pkeyutl -decrypt -inkey B/privkey_B.pem -in A/sympass.enc -out B/sympass.pem

10. Alicja szyfruje plik, który chce przekazać Bobowi za pomocą hasła **sympass.pem** i wysyła zaszyfrowany plik do Boba.


In [14]:
!openssl enc -aes-256-cbc -pbkdf2 -pass file:A/sympass.pem -in bajka.txt -out A/bajka.enc


11. Bob deszyfruje otrzymany plik za pomocą **sympass.pem**.

In [15]:

!openssl enc -d -aes-256-cbc -pbkdf2 -pass file:B/sympass.pem -in A/bajka.enc -out B/bajka.txt
!cat B/bajka.txt

Ludzie prości i zbrojni, więc smutni
Wchodzili na okręty omszałe
Niebo grało podobne do lutni
Srebrnej chyba i kwiatem pachniało
Od uliczek rzeźbionych w cieniu
Szła procesja, czy biały świt
I świergotał jak ptak na ramieniu
Żywot mądry rosnących lip
Potem morza dzieliły się, tarły
Szorstką skórą bokiem o bok
Coś wchodziło, a potem marło
Nie odgadnąć, przez dzień czy rok
Gwiazdy były nisko jak gołębie
Ludzie smutni nachylali twarz
I szukali gwiazd prawdziwych w głębi
Z tym uśmiechem, który chyba znasz
Gwiazdy były nisko jak gołębie
Ludzie smutni nachylali twarz
I szukali gwiazd prawdziwych w głębi
Z tym uśmiechem, który chyba znasz
Aż wyrosły krzepkie i jasne
Jakby wody przezroczysty płaszcz
Więc patrzyli jak na serca własne
Z tym uśmiechem, który chyba znasz
Aż wyrosły krzepkie i jasne
Jakby wody przezroczysty płaszcz
Więc patrzyli jak na serca własne
Z tym uśmiechem, który chyba znasz
Potem lądy się otwarły jak ramy
Góry mrucząc prowadziły pod obłok
Więc rzucali w fale nieba kamień
Ż